In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [3]:
%%sql
drop view if exists seq;
create view seq as (
    select num
    from (
        values (1), (3), (4), (6), (7), (8), (11)
    ) as v(num)
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.


[]

## やりたいこと
+ 数列と1つ後にずれた数列を表示する

In [181]:
%%sql
-- selectの中にサブクエリを書く
select num, (
    select min(num)
    from seq as s2
    where s1.num < s2.num
)
from seq as s1

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.


num,min
1,3
3,4
4,6
6,7
7,8
8,11
11,None


In [4]:
%%sql
-- where句にサブクエリを埋める
select s1.num, s2.num
from seq as s1, seq as s2
where s2.num = (
    select min(num)
    from seq as s3
    where s1.num < s3.num
)

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,num_1
1,3
3,4
4,6
6,7
7,8
8,11


In [175]:
%%sql
-- cross joinで
select s1.num, min(s2.num)
from seq as s1, seq as s2
where s1.num < s2.num
group by s1.num
order by num

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,min
1,3
3,4
4,6
6,7
7,8
8,11


In [179]:
%%sql
-- inner joinで
select s1.num, min(s2.num)
from seq as s1
inner join seq as s2
on s1.num < s2.num
group by s1.num
order by num

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,min
1,3
3,4
4,6
6,7
7,8
8,11


In [177]:
%%sql
-- window関数で(その2)
select num,
    lead(num) over (order by num) as nxt
from seq

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.


num,nxt
1,3
3,4
4,6
6,7
7,8
8,11
11,None


In [178]:
%%sql
-- window関数で(その2)
select num,
    max(num) over (
        order by num
        rows between 1 following and 1 following
    )
from seq

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.


num,max
1,3
3,4
4,6
6,7
7,8
8,11
11,None


+ [s1, s2]を操作するs3を用意する

In [4]:
%%sql
-- cross joinで
select s1.num, s2.num
from seq as s1, seq as s2
where s1.num < s2.num and
not exists (
    select *
    from seq s3
    where s3.num > s1.num and s3.num < s2.num
)

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,num_1
1,3
3,4
4,6
6,7
7,8
8,11


In [5]:
%%sql
-- havingで
select s1.num, s2.num
from seq as s1, seq as s2, seq as s3
where s1.num < s2.num and
s1.num <= s3.num and s3.num <= s2.num
group by s1.num, s2.num
having count(s3.num) = 2
order by s1.num

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,num_1
1,3
3,4
4,6
6,7
7,8
8,11


In [6]:
%%sql
-- s3で[s1.num, s2.num]の数をカウントする
select s1.num, s2.num
from seq as s1, seq as s2
where s1.num < s2.num and
2 = (
    select count(*)
    from seq s3
    where s3.num between s1.num and s2.num
)

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


num,num_1
1,3
3,4
4,6
6,7
7,8
8,11
